# Working with APIs

**Project Goal**: Obatining data with APIs and transforming it to a manageable format

Specifics:

1. Extract stock data from the AlphaVantage API using a URL.
2. Extract stock data from AlphaVantage API using an HTTP request.
3.  Write a function for transforming stock data.
4.  Incorporate Python Exceptions into the function.

In [2]:
import pandas as pd 
import requests

### 1. Accessing APIs Through a URL

- Idenitify components of a URL
- Add API key to config module
- Incorporate AlphaVantage paramters into URL

Identify components of a URL

Notice that this URL has several components. Let's break them down one-by-one.

| URL | Component |
|:--- | :-------- |
| `https://www.alphavantage.co` | This is the **hostname** or **base URL**. It is the web address for the server where we can get our stock data. |
| `/query` | This is the **path**. Most APIs have lots of different operations they can do. The path is the name of the particular operation we want to access. |
| `?` |  This question mark denotes that everything that follows in the URL is a **parameter**. Each parameter is separated by a `&` character. These parameters provide additional information that will change the operation's behavior. This is similar to the way we pass **arguments** into functions in Python. |
| `function=TIME_SERIES_DAILY` | Our first parameter uses the `function` keyword. The value is `TIME_SERIES_DAILY`. In this case, we're asking for **daily** stock data. |
| `symbol=IBM` | Our second parameter uses the `symbol` keyword. So we're asking for a data on a stock whose [**ticker symbol**](https://en.wikipedia.org/wiki/Ticker_symbol) is `IBM`. |
| `apikey=demo` | Much in the same way you need a password to access some websites, an **API key** or **API token** is the password that you'll use to access the API. |

In [4]:
url = ("https://www.alphavantage.co/query?"
       "function=TIME_SERIES_DAILY&"
       "symbol=AMBUJACEM.BSE"
       "apikey=apikey")

print("url type:", type(url))
url

url type: <class 'str'>


'https://www.alphavantage.co/query?function=TIME_SERIES_DAILY&symbol=AMBUJACEM.BSEapikey=apikey'

In [ ]:
# Import settings
from config import settings

### 2. Accessing APIs through an HTTP request

- Defining an HTTP Request
- Make a get request to AlphaVantage API
- Examine contents of API response

### 3. Defensive Programming for APIs

- Create get_daily function
- Raise Exceptions for bad requests 

In [12]:
def get_daily(ticker, output_size="full"):

    """Get daily time series of an equity from AlphaVantage API.

    Parameters
    ----------
    ticker : str
        The ticker symbol of the equity.
    output_size : str, optional
        Number of observations to retrieve. "compact" returns the
        latest 100 observations. "full" returns all observations for
        equity. By default "full".

    Returns
    -------
    pd.DataFrame
        Columns are 'open', 'high', 'low', 'close', and 'volume'.
        All are numeric.
    """
    # Create URL
    url = ("https://www.alphavantage.co/query?"
       "function=TIME_SERIES_DAILY&"
       f"symbol={ticker}&"
       f"outputsize={output_size}&"
       "datatype=json&"
       f"apikey={apikey}")
    
    
    # Send request to API
    response = requests.get(url=url)

    # Extract JSON data from response
    response_data = response.json()
    if 'Time Series (Daily)' not in response_data.keys():
        raise Exception (
            f"Invalid API Call.Check that ticker symbol'{ticker}' is correct"
        )
    
    # Read data into DataFrame
    stock_data = response_data["Time Series (Daily)"]
    df = pd.DataFrame.from_dict(stock_data, orient="index", dtype=float)
    
    # Convert index to `DatetimeIndex` named "date"
    df.index = pd.to_datetime(df.index)
    df.index.name = "date"

    # Remove numbering from columns
    df.columns = [c.split(". ")[1] for c in df.columns]

    # Return DataFrame
    return df

In [15]:
# Test function
df_ambuja = get_daily(ticker="AMBUJACEM.BSE")

print(df_ambuja.info())
df_ambuja.head()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 4767 entries, 2024-05-09 to 2005-01-03
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   open    4767 non-null   float64
 1   high    4767 non-null   float64
 2   low     4767 non-null   float64
 3   close   4767 non-null   float64
 4   volume  4767 non-null   float64
dtypes: float64(5)
memory usage: 223.5 KB
None


,open,high,low,close,volume
date,,,,,
2024-05-09,595.35,596.00,571.95,574.30,115025.0
2024-05-08,590.15,600.85,587.65,594.35,66462.0
2024-05-07,609.95,612.35,589.00,593.55,101391.0
2024-05-06,626.95,626.95,599.70,605.95,110232.0
2024-05-03,630.15,634.75,616.55,622.25,105089.0
